In [ ]:
# Standard imports
from scipy.io import loadmat
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
focused = pd.read_csv("data/focused_revised.csv")
unfocused = pd.read_csv("data/unfocused_revised.csv")

In [16]:
focused.head()

,electrode_a,electrode_b,state
0,2.655329,13.912428,1
1,12.835337,23.628688,1
2,32.473551,50.671904,1
3,27.409727,13.089650,1
4,6.844017,12.315491,1


In [17]:
focused = focused.iloc[:120390]

In [18]:
focused.loc[120389]

electrode_a   -35.949788
electrode_b    27.516314
state           1.000000
Name: 120389, dtype: float64

In [19]:
print(focused.isnull().sum().sum())

0


In [20]:
unfocused = unfocused.iloc[:120399]

In [21]:
unfocused.loc[120398]

electrode_a   -41.079061
electrode_b    30.126764
state           0.000000
Name: 120398, dtype: float64

In [22]:
print(unfocused.isnull().sum().sum())

0


In [23]:
unfocused.tail()

,electrode_a,electrode_b,state
120394,-48.815044,32.258507,0
120395,-37.754290,45.587511,0
120396,-99.129788,-19.991635,0
120397,-92.031458,0.282362,0
120398,-41.079061,30.126764,0


In [24]:
data = pd.concat([focused, unfocused], ignore_index=True)

In [25]:
data.tail()

,electrode_a,electrode_b,state
240784,-48.815044,32.258507,0
240785,-37.754290,45.587511,0
240786,-99.129788,-19.991635,0
240787,-92.031458,0.282362,0
240788,-41.079061,30.126764,0


In [26]:
print(data.isnull().sum().sum())  # no more null values in the data

0


In [15]:
data = data.drop("Unnamed: 0", axis=1)

KeyError: "['Unnamed: 0'] not found in axis"

In [27]:
data.head()

,electrode_a,electrode_b,state
0,2.655329,13.912428,1
1,12.835337,23.628688,1
2,32.473551,50.671904,1
3,27.409727,13.089650,1
4,6.844017,12.315491,1


In [221]:
# for i in range(len(data)):
#     if data.loc[i, "state"] == "unfocused":
#         data.loc[i, "state"] = 0
#     if data.loc[i, "state"] == "focused":
#         data.loc[i, "state"] = 1

for i in range(len(data)):
    if data.loc[i, "state"] == "focused":
        data.loc[i, "state"] = 1

    if data.loc[i, "state"] == "unfocused":
        data.loc[i, "state"] = 0

    if data.loc[i, "state"] == "drowsy":
        data.loc[i, "state"] = 2
    

In [222]:
data.head()

,alpha,delta,beta,gamma,state,channel,subject
0,0.109188,0.121637,0.179848,0.041091,1,AF3,subject_5
1,0.083879,0.078891,0.074366,0.038357,0,AF3,subject_5
2,0.035458,0.109983,0.039020,0.016242,2,AF3,subject_5
3,1.927219,36.255531,0.892434,0.213194,1,F7,subject_5
4,1.758539,36.506171,0.590528,0.163570,0,F7,subject_5


In [223]:
data = data.drop(data[data['state'] == 2].index)

In [224]:
data.head()

,alpha,delta,beta,gamma,state,channel,subject
0,0.109188,0.121637,0.179848,0.041091,1,AF3,subject_5
1,0.083879,0.078891,0.074366,0.038357,0,AF3,subject_5
3,1.927219,36.255531,0.892434,0.213194,1,F7,subject_5
4,1.758539,36.506171,0.590528,0.163570,0,F7,subject_5
6,3.905914,208.404241,0.784781,0.278974,1,F3,subject_5


In [225]:
X = data.drop("state", axis=1)
X = X.drop("channel", axis=1)
X = X.drop("subject", axis=1)

y = data["state"]

In [226]:
X.head()

,alpha,delta,beta,gamma
0,0.109188,0.121637,0.179848,0.041091
1,0.083879,0.078891,0.074366,0.038357
3,1.927219,36.255531,0.892434,0.213194
4,1.758539,36.506171,0.590528,0.163570
6,3.905914,208.404241,0.784781,0.278974


In [227]:
y.head()

0    1
1    0
3    1
4    0
6    1
Name: state, dtype: object

In [28]:
X = data.drop("state", axis=1)
y = data["state"]

In [29]:
# 3. Fit the model to the training data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## Tensorflow Model Using Neural Networks

In [239]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score

In [240]:
model = Sequential()
model.add(Dense(units=32, activation="relu", input_dim=len(X_train.columns)))
model.add(Dense(units=64, activation="relu"))
model.add(Dense(units=1, activation="sigmoid"))

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [231]:
model.compile(loss="binary_crossentropy", optimizer="sgd", metrics="accuracy")

In [232]:
X_train

,alpha,delta,beta,gamma
58,3.450140,622.659452,1.491437,0.240932
144,4.874493,65.714741,1.517355,0.206002
199,0.016487,0.103973,0.013932,0.003655
72,0.086619,0.554427,0.056562,0.035775
196,0.175155,0.503088,0.090317,0.030040
...,...,...,...,...
0,0.109188,0.121637,0.179848,0.041091
117,0.004998,0.154059,0.004049,0.002432
145,13.777919,26.924781,1.188555,0.180265
195,0.238155,0.345524,0.189406,0.049675


In [233]:
model.fit(X_train, y_train, epochs=200, batch_size=32)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

## Scikit-learn

In [30]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [31]:
model.score(X_test, y_test)

0.8480210972216454

In [32]:
X.dtypes

electrode_a    float64
electrode_b    float64
dtype: object

In [33]:
y

0         1
1         1
2         1
3         1
4         1
         ..
240784    0
240785    0
240786    0
240787    0
240788    0
Name: state, Length: 240789, dtype: int64

In [34]:
X.shape

(240789, 2)

In [35]:
X = X.to_numpy(dtype='float32')  # Ensure float data type
y = y.to_numpy(dtype='int32') 

In [36]:
model.score(X_train, y_train)

0.9999377047308066